In [ ]:
import tensorflow as tf, pathlib, zipfile, json, os
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

ZIP_PATH      = "gesture_data.zip"   # <-- upload this file
DATA     = pathlib.Path("gesture_data")
IMG_SIZE      = (224, 224)
BATCH_SIZE    = 32
EPOCHS_HEAD   = 8                         # warm-up head
EPOCHS_FINE   = 6                         # fine-tune last block
PATIENCE      = 3                         # for early stopping

# Unzip dataset
zipfile.ZipFile(ZIP_PATH).extractall(DATA)

# Count classes automatically
DATA_ROOT     = pathlib.Path("gesture_data/gesture_data")
class_names = sorted([p.name for p in DATA_ROOT.iterdir() if p.is_dir()])
num_classes = len(class_names)
assert num_classes in (2,3), f"Expected 2 or 3 folders, found {num_classes}: {class_names}"
print("Classes:", class_names)

Classes: ['background', 'thumbs_up', 'v_sign']


In [ ]:
datagen = ImageDataGenerator(
    rescale=1/255.,
    validation_split=0.2,
    rotation_range=15,
    width_shift_range=0.10,
    height_shift_range=0.10,
    brightness_range=[0.7, 1.3],
    zoom_range=0.10
)

train_gen = datagen.flow_from_directory(
    DATA_ROOT,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    DATA_ROOT,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='validation'
)


Found 360 images belonging to 3 classes.
Found 90 images belonging to 3 classes.


In [ ]:
base = tf.keras.applications.VGG16(
    include_top=False, weights="imagenet",
    input_shape=IMG_SIZE + (3,))
base.trainable = False

model = models.Sequential([
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.30),
    layers.Dense(num_classes, activation="softmax")
])

model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

es = EarlyStopping(patience=PATIENCE, monitor="val_accuracy",
                   restore_best_weights=True, verbose=1)
ck = ModelCheckpoint("best_head.keras", monitor="val_accuracy",
                     save_best_only=True, verbose=1)

history_head = model.fit(train_gen, validation_data=val_gen,
                         epochs=EPOCHS_HEAD, callbacks=[es, ck])


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.3568 - loss: 1.1435 

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: val_accuracy improved from -inf to 0.60000, saving model to best_head.keras
12/12 ━━━━━━━━━━━━━━━━━━━━ 189s 16s/step - accuracy: 0.3574 - loss: 1.1426 - val_accuracy: 0.6000 - val_loss: 1.0158
Epoch 2/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.5279 - loss: 1.0042 
Epoch 2: val_accuracy improved from 0.60000 to 0.66667, saving model to best_head.keras
12/12 ━━━━━━━━━━━━━━━━━━━━ 184s 16s/step - accuracy: 0.5294 - loss: 1.0036 - val_accuracy: 0.6667 - val_loss: 0.9385
Epoch 3/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.5999 - loss: 0.9807 
Epoch 3: val_accuracy did not improve from 0.66667
12/12 ━━━━━━━━━━━━━━━━━━━━ 185s 16s/step - accuracy: 0.5999 - loss: 0.9794 - val_accuracy: 0.6556 - val_loss: 0.8647
Epoch 4/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.6645 - loss: 0.8887 
Epoch 4: val_accuracy improved from 0.66667 to 0.67778, saving model to best_head.keras
12/12 ━━━━━━━━━━━━━━━━━━━━ 183s 15s/step - accuracy: 0.6640 - loss: 0.8878 - val_acc

In [ ]:
base.trainable = True
for layer in base.layers[:-4]:  # freeze all but block5_conv*
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

ck_ft = ModelCheckpoint("best_finetune.keras", monitor="val_accuracy",
                        save_best_only=True, verbose=1)
es_ft = EarlyStopping(patience=PATIENCE, monitor="val_accuracy",
                      restore_best_weights=True, verbose=1)

history_ft = model.fit(train_gen, validation_data=val_gen,
                       epochs=EPOCHS_FINE, callbacks=[es_ft, ck_ft])

Epoch 1/6
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - accuracy: 0.7243 - loss: 0.7211 
Epoch 1: val_accuracy improved from -inf to 0.87778, saving model to best_finetune.keras
12/12 ━━━━━━━━━━━━━━━━━━━━ 222s 18s/step - accuracy: 0.7267 - loss: 0.7178 - val_accuracy: 0.8778 - val_loss: 0.5539
Epoch 2/6
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - accuracy: 0.8014 - loss: 0.5557 
Epoch 2: val_accuracy improved from 0.87778 to 0.90000, saving model to best_finetune.keras
12/12 ━━━━━━━━━━━━━━━━━━━━ 218s 18s/step - accuracy: 0.8023 - loss: 0.5535 - val_accuracy: 0.9000 - val_loss: 0.4533
Epoch 3/6
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - accuracy: 0.8647 - loss: 0.4422 
Epoch 3: val_accuracy improved from 0.90000 to 0.93333, saving model to best_finetune.keras
12/12 ━━━━━━━━━━━━━━━━━━━━ 218s 18s/step - accuracy: 0.8638 - loss: 0.4414 - val_accuracy: 0.9333 - val_loss: 0.3728
Epoch 4/6
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - accuracy: 0.8798 - loss: 0.3863 
Epoch 4: val_accuracy improved from 0.93

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("gestures.tflite", "wb").write(tflite_model)

print("\n✅  Training complete.  Validation accuracy:",
      history_ft.history.get("val_accuracy", history_head.history["val_accuracy"])[-1])
print("📦  Saved gestures.tflite  (download from the left-side file browser)")

Saved artifact at '/tmp/tmp1nifcntu'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_43')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  132638799622800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132638799621648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132638799622032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132638799620880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132638799621456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132638799620112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132638799620688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132638799619344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132638799619920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132638799618576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13263879961